In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
%env PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON

from posydon.popsyn.binarypopulation import BinaryPopulation
from posydon.binary_evol import SimulationProperties
from posydon.binary_evol.flow_chart import flow_chart
from posydon.binary_evol.MESA.step_mesa import CO_HeMS_step, MS_MS_step, CO_HMS_RLO_step
from posydon.binary_evol.DT.step_detached import detached_step
from posydon.binary_evol.CE.step_CEE import StepCEE
from posydon.binary_evol.SN.step_SN import StepSN
from posydon.binary_evol.DT.double_CO import DoubleCO
from posydon.binary_evol.step_end import step_end
from posydon.binary_evol.simulationproperties import TimingHooks, StepNamesHooks

import sys
import numpy as np

env: PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON


In [3]:
sheme = 'IF'

if sheme == 'NN':
    interpolation_method='nearest_neighbour'
    save_initial_conditions=True
    track_interpolation=False
elif sheme == 'IF':
    interpolation_method='linear3c_kNN'
    save_initial_conditions=False
    track_interpolation=False  

sim_kwargs = dict(
    flow = (flow_chart, {}),
    step_HMS_HMS = (MS_MS_step, dict(interpolation_method=interpolation_method,
                                     save_initial_conditions=save_initial_conditions,
                                     track_interpolation=track_interpolation)),
    step_CO_HeMS = (CO_HeMS_step, dict(interpolation_method=interpolation_method,
                                     save_initial_conditions=save_initial_conditions,
                                     track_interpolation=track_interpolation)),
    step_CO_HMS_RLO = (CO_HMS_RLO_step, dict(interpolation_method=interpolation_method,
                                     save_initial_conditions=save_initial_conditions,
                                     track_interpolation=track_interpolation)),
    step_detached = (detached_step, {}),
    step_CE = (StepCEE, dict(core_definition_H_fraction= 0.1,
                            )),
    step_SN = (StepSN, {}), 
    step_dco = (DoubleCO, {}),
    step_end = (step_end, {}),
    extra_hooks = [(TimingHooks, {}),(StepNamesHooks, {})],
    verbose=True
)

sim_prop = SimulationProperties(**sim_kwargs)

kwargs = {'number_of_binaries' : 100,
          'primary_mass_min' : 7,
          'primary_mass_max' : 45,
          'secondary_mass_scheme' : 'flat_mass_ratio',
          'secondary_mass_min': 7,
          'secondary_mass_max': 45,
          'orbital_separation_min': 1,
          'orbital_separation_max': 3e3,
          'eccentricity_scheme':'zero',
          'extra_columns' : ['step_times','step_names'], 
          'only_select_columns' : ['state', 'event', 'time', 'lg_mtransfer_rate',
                                   'orbital_period'],
          #'selection_function' : lambda binary: not(binary in pop.find_failed()),
          'include_S1' : True , 
          'S1_kwargs' : {'only_select_columns' : ['state', 'mass', 
                                  'log_R', 'center_h1','center_he4',
                                  'he_core_mass', 'surface_he4', 'surface_h1',
                                  'lg_mdot'],
                        'scalar_names' : ['natal_kick_array', 'SN_type']},
          'include_S2' : True,
          'S2_kwargs' : {'only_select_columns' : ['state', 'mass', 
                                  'log_R', 'center_h1','center_he4',
                                  'he_core_mass', 'surface_he4', 'surface_h1',
                                  'lg_mdot'],
                        'scalar_names' : ['natal_kick_array', 'SN_type']},
          'star_formation' : 'burst',
          'max_simulation_time' : 13.7e9,
         }

pop = BinaryPopulation(
    population_properties=sim_prop,
#     file_name = 'DBH_CE2_dco.h5',
    **kwargs
)

pop.evolve(breakdown_to_df=False, from_hdf=False, tqdm=True, **kwargs)



100%|██████████| 100/100 [00:32<00:00,  3.06it/s]


In [4]:
pop[2].to_df(**kwargs)[['time','S1_mass','S2_mass','orbital_period', 'S1_log_R', 'S2_log_R' ,'S1_lg_mdot','S2_lg_mdot','step_names','event',
                       'S1_state', 'S2_state', 'S1_surface_he4']]

,time,S1_mass,S2_mass,orbital_period,S1_log_R,S2_log_R,S1_lg_mdot,S2_lg_mdot,step_names,event,S1_state,S2_state,S1_surface_he4
binary_index,,,,,,,,,,,,,
2,0,10.6078,7.14555,11.4324,NaN,NaN,NaN,NaN,initial_cond,ZAMS,H-rich_Core_H_burning,H-rich_Core_H_burning,NaN
2,2.56586e+07,3.35586,7.4039,14.7476,0.551295,0.59554,-5.7074,-8.55132,step_HMS_HMS,CC1,stripped_He_Central_C_depletion,H-rich_Core_H_burning,0.98615
2,2.56586e+07,1.15069,7.4039,NaN,NaN,0.59554,NaN,NaN,step_SN,NaN,NS,H-rich_Core_H_burning,NaN
2,2.56586e+07,1.15069,7.4039,NaN,NaN,0.59554,NaN,NaN,step_end,END,NS,H-rich_Core_H_burning,NaN


In [5]:
pop[0].to_df(**kwargs).columns

Index(['state', 'event', 'time', 'lg_mtransfer_rate', 'orbital_period',
       'step_times', 'step_names', 'S1_state', 'S1_mass', 'S1_log_R',
       'S1_center_h1', 'S1_center_he4', 'S1_he_core_mass', 'S1_surface_he4',
       'S1_surface_h1', 'S1_lg_mdot', 'S2_state', 'S2_mass', 'S2_log_R',
       'S2_center_h1', 'S2_center_he4', 'S2_he_core_mass', 'S2_surface_he4',
       'S2_surface_h1', 'S2_lg_mdot'],
      dtype='object')

In [8]:
%env PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON



env: PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON


In [9]:
import os
PATH_TO_POSYDON = os.environ.get("PATH_TO_POSYDON")

In [10]:
print(PATH_TO_POSYDON)

/projects/b1119/mpg0328/tutorial/POSYDON


In [7]:
os.environ

environ{'PATH': '/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/software/anaconda3.6/bin:/software/qanalytics/jupyterhub/env/bin',
        'LANG': 'en_US.UTF-8',
        'JUPYTERHUB_API_TOKEN': '388ada75755f436e9101fd1b813df54b',
        'JPY_API_TOKEN': '388ada75755f436e9101fd1b813df54b',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-mpg0328',
        'JUPYTERHUB_HOST': '',
        'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/mpg0328/oauth_callback',
        'JUPYTERHUB_USER': 'mpg0328',
        'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
        'JUPYTERHUB_BASE_URL': '/',
        'JUPYTERHUB_SERVICE_PREFIX': '/user/mpg0328/',
        'USER': 'mpg0328',
        'HOME': '/home/mpg0328',
        'SHELL': '/bin/bash',
        'JPY_PARENT_PID': '371894',
        'TERM': 'xterm-color',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://ipykernel.pylab.backend_inline'}

In [4]:
from posydon.binary_evol.singlestar import SingleStar
from posydon.binary_evol.binarystar import BinaryStar

In [5]:
kwargs1 = dict(state='H-rich_Core_H_burning',
           mass=20.0,
           metallicity=0.014)
star_1 = SingleStar(**kwargs1)


kwargs2 = dict(state='H-rich_Core_H_burning',
           mass=10.0,
           metallicity=0.014)
star_2 = SingleStar(**kwargs2)


kwargs3 = {'state' : 'detached',
           'event' : 'ZAMS',
           'time' : 0.,
           'orbital_period' : 100.,
           'eccentricity' : 0.}
binary = BinaryStar(star_1, star_2, properties=sim_prop, **kwargs3)

In [6]:
binary.evolve()

In [9]:
binary.event_history
binary.state_history

['detached', 'detached', 'disrupted', 'disrupted']

In [10]:
binary.star_1.state_history

['H-rich_Core_H_burning', 'stripped_He_Central_C_depletion', 'NS', 'NS']

In [11]:
df_test = binary.to_df()

In [19]:
print(df_test.columns.tolist())

['state', 'event', 'time', 'separation', 'orbital_period', 'eccentricity', 'V_sys', 'rl_relative_overflow_1', 'rl_relative_overflow_2', 'lg_mtransfer_rate', 'mass_transfer_case', 'trap_radius', 'acc_radius', 't_sync_rad_1', 't_sync_conv_1', 't_sync_rad_2', 't_sync_conv_2', 'nearest_neighbour_distance', 'p_classification', 'S1_state', 'S1_metallicity', 'S1_mass', 'S1_log_R', 'S1_log_L', 'S1_lg_mdot', 'S1_lg_system_mdot', 'S1_lg_wind_mdot', 'S1_he_core_mass', 'S1_he_core_radius', 'S1_c_core_mass', 'S1_c_core_radius', 'S1_o_core_mass', 'S1_o_core_radius', 'S1_co_core_mass', 'S1_co_core_radius', 'S1_center_h1', 'S1_center_he4', 'S1_center_c12', 'S1_center_n14', 'S1_center_o16', 'S1_surface_h1', 'S1_surface_he4', 'S1_surface_c12', 'S1_surface_n14', 'S1_surface_o16', 'S1_log_LH', 'S1_log_LHe', 'S1_log_LZ', 'S1_log_Lnuc', 'S1_c12_c12', 'S1_center_gamma', 'S1_avg_c_in_c_core', 'S1_surf_avg_omega', 'S1_surf_avg_omega_div_omega_crit', 'S1_total_moment_of_inertia', 'S1_log_total_angular_momentum'

In [20]:
df_test['S1_state']

binary_index
NaN              H-rich_Core_H_burning
NaN    stripped_He_Central_C_depletion
NaN                                 NS
NaN                                 NS
Name: S1_state, dtype: object